## ICR - Identifying Age-Related Conditions¶


Use Machine Learning to detect conditions with measurements of anonymous characteristics
https://www.kaggle.com/code/samuelabatnehendalie/icr-identifying-age-related-conditions

### Importing Libraries

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sklearn
import sys

print(sys.version)

print(sklearn.__version__)

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler

from sklearn.impute import KNNImputer



3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
1.0.2


In [93]:
''' use this code when upload to kaggle
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
'''

' use this code when upload to kaggle\n# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python\n# For example, here\'s several helpful packages to load\n\nimport numpy as np # linear algebra\nimport pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\n# Input data files are available in the read-only "../input/" directory\n# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory\n\nimport os\nfor dirname, _, filenames in os.walk(\'/kaggle/input\'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n\n# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"\n# You can also write temporary files to /kaggle/temp/, but they wo

In [94]:
directory = "../data"


In [95]:
train = pd.read_csv(os.path.join(directory, "train.csv"))
meta_data = pd.read_csv(os.path.join(directory, "greeks.csv"))
test = pd.read_csv(os.path.join(directory, "test.csv"))

### Exploratory Data Analysis

In [96]:
train.head() # a lot of columns!

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [97]:
train.describe() # may be coming from different distribution


,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
count,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,...,616.000000,617.000000,615.000000,617.000000,617.000000,617.000000,617.000000,617.000000,616.000000,617.000000
mean,0.477149,3502.013221,118.624513,38.968552,10.128242,5.545576,0.060320,10.566447,8.053012,5350.388655,...,5.433199,3.533905,0.421501,20.724856,131.714987,14679.595398,31.489716,50.584437,8.530961,0.175041
std,0.468388,2300.322717,127.838950,69.728226,10.518877,2.551696,0.416817,4.350645,65.166943,3021.326641,...,11.496257,50.181948,1.305365,9.991907,144.181524,19352.959387,9.864239,36.266251,10.327010,0.380310
min,0.081187,192.593280,85.200147,3.177522,8.138688,0.699861,0.025578,3.396778,1.229900,1693.624320,...,0.173229,0.497060,0.067730,4.102182,72.611063,13.038894,9.432735,0.897628,0.001129,0.000000
25%,0.252107,2197.345480,85.200147,12.270314,8.138688,4.128294,0.025578,8.129580,1.229900,4155.702870,...,0.173229,0.497060,0.067730,14.036718,72.611063,2798.992584,25.034888,23.011684,0.124392,0.000000
50%,0.354659,3120.318960,85.200147,20.533110,8.138688,5.031912,0.025578,10.461320,1.229900,4997.960730,...,3.028141,1.131000,0.250601,18.771436,72.611063,7838.273610,30.608946,41.007968,0.337827,0.000000
75%,0.559763,4361.637390,113.739540,39.139886,8.138688,6.431634,0.036845,12.969516,5.081244,6035.885700,...,6.238814,1.512060,0.535067,25.608406,127.591671,19035.709240,36.863947,67.931664,21.978000,0.000000
max,6.161666,28688.187660,1910.123198,630.518230,178.943634,38.270880,10.315851,38.971568,1463.693448,53060.599240,...,137.932739,1244.227020,31.365763,135.781294,1497.351958,143790.071200,81.210825,191.194764,21.978000,1.000000


In [98]:
train.info() # can see that there are several null values in the data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 58 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      617 non-null    object 
 1   AB      617 non-null    float64
 2   AF      617 non-null    float64
 3   AH      617 non-null    float64
 4   AM      617 non-null    float64
 5   AR      617 non-null    float64
 6   AX      617 non-null    float64
 7   AY      617 non-null    float64
 8   AZ      617 non-null    float64
 9   BC      617 non-null    float64
 10  BD      617 non-null    float64
 11  BN      617 non-null    float64
 12  BP      617 non-null    float64
 13  BQ      557 non-null    float64
 14  BR      617 non-null    float64
 15  BZ      617 non-null    float64
 16  CB      615 non-null    float64
 17  CC      614 non-null    float64
 18  CD      617 non-null    float64
 19  CF      617 non-null    float64
 20  CH      617 non-null    float64
 21  CL      617 non-null    float64
 22  CR

In [99]:
train.nunique()

Id       617
AB       217
AF       599
AH       227
AM       605
AR       130
AX       427
AY       148
AZ       484
BC       259
BD       617
BN        53
BP       612
BQ       515
BR       566
BZ       115
CB       553
CC       602
CD       584
CF       586
CH       135
CL       123
CR       595
CS       576
CU       307
CW       426
DA       611
DE       616
DF       137
DH       191
DI       571
DL       604
DN       576
DU       253
DV        39
DY       590
EB       439
EE       513
EG       610
EH       127
EJ         2
EL       311
EP       275
EU       455
FC       600
FD       337
FE       615
FI       498
FL       388
FR       435
FS       161
GB       560
GE       264
GF       611
GH       596
GI       615
GL       355
Class      2
dtype: int64

In [100]:
train.info() # no more null values


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 58 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      617 non-null    object 
 1   AB      617 non-null    float64
 2   AF      617 non-null    float64
 3   AH      617 non-null    float64
 4   AM      617 non-null    float64
 5   AR      617 non-null    float64
 6   AX      617 non-null    float64
 7   AY      617 non-null    float64
 8   AZ      617 non-null    float64
 9   BC      617 non-null    float64
 10  BD      617 non-null    float64
 11  BN      617 non-null    float64
 12  BP      617 non-null    float64
 13  BQ      557 non-null    float64
 14  BR      617 non-null    float64
 15  BZ      617 non-null    float64
 16  CB      615 non-null    float64
 17  CC      614 non-null    float64
 18  CD      617 non-null    float64
 19  CF      617 non-null    float64
 20  CH      617 non-null    float64
 21  CL      617 non-null    float64
 22  CR

In [101]:
ytrain = train['Class']
print(ytrain)
train = train.drop(['Class', 'Id'], axis=1)
train.head()

0      1
1      0
2      0
3      0
4      1
      ..
612    0
613    0
614    0
615    0
616    0
Name: Class, Length: 617, dtype: int64


,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343
1,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000
2,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941
3,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829
4,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614


In [102]:
train['EJ'].unique()

array(['B', 'A'], dtype=object)

In [103]:
# make categorical data into numerical data
train.EJ = train.EJ.replace({'A': 0, 'B': 1})
train.head()

imputer = KNNImputer(n_neighbors=5)
train = pd.DataFrame(imputer.fit_transform(train), columns=train.columns)

train.head()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343
1,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000
2,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941
3,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829
4,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614


In [104]:
correlation_matrix = train.corr()
threshold = 0.9
highly_correlated_features = set()

# Iterate through the correlation matrix
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        # Check if the correlation exceeds the threshold
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            # Add the correlated feature to the set
            feature_name = correlation_matrix.columns[i]
            highly_correlated_features.add(feature_name)

train = train.drop(highly_correlated_features, axis=1)
train.head()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI
0,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,9028.291921,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944
1,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,6785.003474,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996
2,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,8338.906181,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676
3,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,10965.766040,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248
4,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,16198.049590,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628


In [105]:
 # referenced https://www.kaggle.com/code/dan3dewey/icr-2023-balanced-log-loss
def competition_log_loss(y_true, y_pred): # use predict proba
    # y_true: correct labels 0, 1
    # y_pred: predicted probabilities of class=1
    # Implements the Evaluation equation with w_0 = w_1 = 1.
    # Calculate the number of observations for each class
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    print(y_pred.shape)
    # Calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred[:,1], 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    # Calculate the average log loss for each class
    #print("reached")
    print(p_0.shape, p_1.shape, y_true.shape)

    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0)) / N_0
    log_loss_1 = -np.sum(y_true * np.log(p_1)) / N_1
    # return the (not further weighted) average of the averages
    return (log_loss_0 + log_loss_1)/2


def competition_log_loss2(y_true, y_pred): # for grid search
    # y_true: correct labels 0, 1
    # y_pred: predicted probabilities of class=1
    # Implements the Evaluation equation with w_0 = w_1 = 1.
    # Calculate the number of observations for each class
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    print(y_pred.shape)
    # Calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    # Calculate the average log loss for each class
    #print("reached")
    print(p_0.shape, p_1.shape, y_true.shape)

    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0)) / N_0
    log_loss_1 = -np.sum(y_true * np.log(p_1)) / N_1
    # return the (not further weighted) average of the averages
    return (log_loss_0 + log_loss_1)/2

### Train Test Split

In [106]:
# train test split such that same distribution of classes in train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, ytrain, test_size=0.2, random_state=42, stratify=ytrain)

### Scale the Variables


In [107]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [108]:
import optuna
from xgboost import XGBClassifier

def objectiveXGB(trial):
    # Define the search space for hyperparameters
    param_space = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'max_depth': trial.suggest_int('max_depth', 2, 7),
        'n_estimators': trial.suggest_categorical('n_estimators', [50,100,150, 200,250, 300])
    }

    # Create an XGBoost classifier with the suggested hyperparameters
    xgb_model = XGBClassifier(**param_space)

    # Train and evaluate the model
    xgb_model.fit(X_train, y_train)
    y_pred = xgb_model.predict_proba(X_test)
    score = competition_log_loss(y_test, y_pred)

    return score

studyXGB = optuna.create_study(direction='minimize')
studyXGB.optimize(objectiveXGB, n_trials=100)
best_paramsXGB = studyXGB.best_params



[I 2023-05-15 22:46:13,747] A new study created in memory with name: no-name-c7a880d3-7ad3-4ef3-84a0-dccbc76fe5be
/Users/gary/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  import sys
[I 2023-05-15 22:46:14,122] Trial 0 finished with value: 0.5489261388486507 and parameters: {'learning_rate': 0.018997834976405212, 'max_depth': 2, 'n_estimators': 100}. Best is trial 0 with value: 0.5489261388486507.


(124, 2)
(124,) (124,) (124,)


[I 2023-05-15 22:46:15,212] Trial 1 finished with value: 0.5565386386786262 and parameters: {'learning_rate': 0.056360868265538655, 'max_depth': 5, 'n_estimators': 250}. Best is trial 0 with value: 0.5489261388486507.


(124, 2)
(124,) (124,) (124,)


[I 2023-05-15 22:46:15,961] Trial 2 finished with value: 0.5115809163307761 and parameters: {'learning_rate': 0.004911867263260194, 'max_depth': 4, 'n_estimators': 150}. Best is trial 2 with value: 0.5115809163307761.


...

(124, 2)
(124,) (124,) (124,)


[I 2023-05-15 22:48:08,975] Trial 98 finished with value: 0.4527814266240347 and parameters: {'learning_rate': 0.01802600422218797, 'max_depth': 6, 'n_estimators': 150}. Best is trial 77 with value: 0.4461993643144102.


(124, 2)
(124,) (124,) (124,)


[I 2023-05-15 22:48:10,835] Trial 99 finished with value: 0.5279697959315768 and parameters: {'learning_rate': 0.024599444833916143, 'max_depth': 7, 'n_estimators': 300}. Best is trial 77 with value: 0.4461993643144102.


(124, 2)
(124,) (124,) (124,)


In [109]:
print("Best params:", best_paramsXGB)
print(studyXGB.best_value)

Best params: {'learning_rate': 0.016765214601525125, 'max_depth': 7, 'n_estimators': 150}
0.4461993643144102


In [110]:
from lightgbm import LGBMClassifier


# optimize lightgbm
def objectiveLGBM(trial):
    # Define the search space for hyperparameters
    param_space = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'max_depth': trial.suggest_int('max_depth', 2, 7),
        'n_estimators': trial.suggest_categorical('n_estimators', [50,100,150, 200,250, 300])
    }

    # Create an XGBoost classifier with the suggested hyperparameters
    lgbm_model = LGBMClassifier(**param_space)

    # Train and evaluate the model
    lgbm_model.fit(X_train, y_train)
    y_pred = lgbm_model.predict_proba(X_test)
    score = competition_log_loss(y_test, y_pred)

    return score

studyLGBM = optuna.create_study(direction='minimize')
studyLGBM.optimize(objectiveLGBM, n_trials=100)
best_paramsLGBM = studyLGBM.best_params



[I 2023-05-15 22:48:11,188] A new study created in memory with name: no-name-2e2f8d75-1f71-4952-ae54-3750bb2dd06b
/Users/gary/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  
[I 2023-05-15 22:48:12,809] Trial 0 finished with value: 0.46646256332455394 and parameters: {'learning_rate': 0.03232993025696738, 'max_depth': 5, 'n_estimators': 200}. Best is trial 0 with value: 0.46646256332455394.


(124, 2)
(124,) (124,) (124,)


/Users/gary/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  
[I 2023-05-15 22:48:14,234] Trial 1 finished with value: 0.5413936592258555 and parameters: {'learning_rate': 0.05790340143427641, 'max_depth': 6, 'n_estimators': 150}. Best is trial 0 with value: 0.46646256332455394.


(124, 2)
(124,) (124,) (124,)


/Users/gary/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  
[I 2023-05-15 22:48:14,510] Trial 2 finished with value: 0.7170690797903845 and parameters: {'learning_rate': 0.003622426290021295, 'max_depth': 2, 'n_estimators': 200}. Best is trial 0 with value: 0.46646256332455394.


...

(124, 2)
(124,) (124,) (124,)


/Users/gary/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  
[I 2023-05-15 22:49:46,306] Trial 98 finished with value: 0.4712424580767318 and parameters: {'learning_rate': 0.05132414381011191, 'max_depth': 4, 'n_estimators': 200}. Best is trial 63 with value: 0.43311739336041916.


(124, 2)
(124,) (124,) (124,)


/Users/gary/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  
[I 2023-05-15 22:49:46,948] Trial 99 finished with value: 0.46624750603236936 and parameters: {'learning_rate': 0.046232820439526916, 'max_depth': 3, 'n_estimators': 150}. Best is trial 63 with value: 0.43311739336041916.


(124, 2)
(124,) (124,) (124,)


In [111]:
print(best_paramsLGBM)
print(studyLGBM.best_value)


{'learning_rate': 0.06722429993159512, 'max_depth': 3, 'n_estimators': 150}
0.43311739336041916


In [112]:
# ensemble
from sklearn.ensemble import VotingClassifier

xgb = XGBClassifier(**best_paramsXGB)
lgbm = LGBMClassifier(**best_paramsLGBM)

estimators = [('xgb', xgb), ('lgbm', lgbm)]

ensemble = VotingClassifier(estimators, voting='soft')
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_train)

print('Accuracy of ensemble classifier on train set: {:.2f}'.format(accuracy_score(y_train, y_pred)))
print(confusion_matrix(y_train, y_pred))
print(classification_report(y_train, y_pred))

y_pred = ensemble.predict(X_test)
y_pred2 =  ensemble.predict_proba(X_test)

print('Accuracy of ensemble classifier on test set: {:.2f}'.format(accuracy_score(y_test, y_pred)))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

print(competition_log_loss(y_test, y_pred2))


Accuracy of ensemble classifier on train set: 1.00
[[407   0]
 [  0  86]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       407
           1       1.00      1.00      1.00        86

    accuracy                           1.00       493
   macro avg       1.00      1.00      1.00       493
weighted avg       1.00      1.00      1.00       493

Accuracy of ensemble classifier on test set: 0.91
[[98  4]
 [ 7 15]]
              precision    recall  f1-score   support

           0       0.93      0.96      0.95       102
           1       0.79      0.68      0.73        22

    accuracy                           0.91       124
   macro avg       0.86      0.82      0.84       124
weighted avg       0.91      0.91      0.91       124

(124, 2)
(124,) (124,) (124,)
0.4206964003542287


### Test data

In [113]:
test = pd.read_csv(os.path.join(directory, "test.csv"))
test.head()


,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,00eed32682bb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,010ebe33f668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,02fa521e1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,040e15f562a2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,046e85c7cc7f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [114]:
# remove BQ and EL since they have more 50 null values

# impute null values


# drop Id column
test_id = test['Id']
test = test.drop(['Id'], axis=1)

# make categorical data into numerical data
test.EJ = test.EJ.replace({'A': 0, 'B': 1})
test = pd.DataFrame(imputer.transform(test), columns=test.columns)

# remove highly correlated features
test = test.drop(highly_correlated_features, axis=1)


test.head()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [115]:
# predict


test = scaler.transform(test)

In [116]:
xgb = XGBClassifier(**best_paramsXGB)
lgbm = LGBMClassifier(**best_paramsLGBM)

estimators = [('xgb', xgb), ('lgbm', lgbm)]

ensemble = VotingClassifier(estimators, voting='soft')
ensemble.fit(train, ytrain)
y_prob = ensemble.predict_proba(test)


In [117]:
out = pd.DataFrame(
    {
        "Id": test_id,
        "class_0": y_prob[:,0],
        "class_1": y_prob[:,1],
    }
)

In [118]:
out

,Id,class_0,class_1
0,00eed32682bb,0.821006,0.178994
1,010ebe33f668,0.821006,0.178994
2,02fa521e1838,0.821006,0.178994
3,040e15f562a2,0.821006,0.178994
4,046e85c7cc7f,0.821006,0.178994


In [119]:
out.to_csv(r"submission.csv", index=False)
